In [1]:
# read../visualization/assets/data/matrix/new_community_evolution.csv

import os
import pandas as pd
import json
import numpy as np

handle_df = pd.read_csv("../visualization/assets/data/matrix/new_community_evolution.csv")
# # handle, 320
import numpy as np

RAD = np.pi / 180

# addprocessNaN
def handle_nan(obj):
 """processJSONcolumnNaN, null"""
 if isinstance(obj, dict):
 return {k: handle_nan(v) for k, v in obj.items()}
 elif isinstance(obj, list):
 return [handle_nan(i) for i in obj]
 elif isinstance(obj, float) and np.isnan(obj):
 return None
 else:
 return obj

def polarToCartesian(asrank, attr, radius=350):
 """
 xy
 :param asrank: processasrankdata, DataFrameformat, longitude, attrfield
 :param attr: as, cone
 :param radius: node
 :return:
 """
 # mappingconemin-max mapping 0-350
 min_cone = asrank[attr].min()
 max_cone = asrank[attr].max()

 # cone, ,  max_Domain = 1 - np.log((min_cone + 1) / (max_cone + 1))

 # conelogprocess
 asrank['cone_map'] = asrank[attr].apply(lambda x: (1 - np.log((x + 1) / (max_cone + 1))))

 # logprocess, cone0-350mapping
 asrank['r'] = asrank['cone_map'].apply(lambda x: (((x - 1) / (max_Domain - 1)) * radius))

 # xy x=+rcosθ
 x = 1900 / 2 + asrank['r'] * np.cos(asrank['longitude'] * RAD)
 y = 1000 / 2 - asrank['r'] * np.sin(asrank['longitude'] * RAD)

 asrank['x'] = x
 asrank['y'] = y
 # rank rfield
 asrank.sort_values(by=attr, ascending=False, inplace=True)
 asrank.drop_duplicates(subset=['id', attr], keep='last', inplace=True)

 return asrank


# for date in range(202404, 202411):
# cord_rank = pd.read_csv(f'./{date}/handle/rank{date}.csv')
# cord_rank1 = pd.read_csv(f'./{date}/rank{date}.csv')
# cord_rank = polarToCartesian(cord_rank, 'cone', 320)
# cord_rank1 = polarToCartesian(cord_rank1, 'cone', 320)
# cord_rank.fillna(0, inplace=True)
# cord_rank1.fillna(0, inplace=True)
# cord_rank.to_csv(f'./{date}/handle/rank{date}.csv', index=False)
# cord_rank1.to_csv(f'./{date}/rank{date}.csv', index=False)


base_dir = './'

# Initialize rankjson and reljson
rankjson = {}
reljson = {}

# Get all unique values of each column in handle_df
for col in handle_df.columns:
 unique_values = handle_df[col].unique() # columnunique values
 rankjson[col] = {} # createcolumn rankjson  reljson[col] = {} # createcolumn reljson 
 # columnunique values
 for value in unique_values:
 # value , JSON  value_str = str(value)

 # Get rank and rel file paths by column name
 rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
 rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

 # read rank rel file
 temp_rank = pd.read_csv(rank_path)
 temp_rel = pd.read_csv(rel_path)

 # Filter data by conditions
 filtered_rank = temp_rank[temp_rank['community'] == value]
 filtered_rel = temp_rel[temp_rel['type'] == str(value)]
 filtered_rank = polarToCartesian(filtered_rank, 'cone', 320)

 if len(filtered_rank) != 0 and len(filtered_rel) != 0:
 filtered_rel['cone'] = filtered_rel.apply(
 lambda row: max(
 filtered_rank.loc[filtered_rank['id'] == row['source'], 'cone'].iloc[0], # source cone  filtered_rank.loc[filtered_rank['id'] == row['target'], 'cone'].iloc[0] # target cone  ),
 axis=1
 )

 os.makedirs(f'./local_core/{col}/raw', exist_ok=True)
 # filter rankjson reljson  # NaNNone, JSONcolumn
 rankjson[col][value_str] = handle_nan(filtered_rank.replace({np.nan: None}).to_dict(orient='records'))
 reljson[col][value_str] = handle_nan(filtered_rel.replace({np.nan: None}).to_dict(orient='records'))

 with open(f'./local_core/{col}/raw/rank.json', 'w', encoding='utf-8') as rank_file:
 json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

 with open(f'./local_core/{col}/raw/rel.json', 'w', encoding='utf-8') as rel_file:
 json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
 print(f"Saved data for {col}")


# node, , relation, export
def process_rank_and_rel(t1, t2):
 t1.fillna(0, inplace=True)
 t1_agg = t1.loc[t1.groupby('country')['cone'].idxmax()]

 t1_agg['cone'] = t1_agg['country'].apply(
 lambda country: t1[t1['country'] == country]['cone'].sum()
 )
 t1_agg = polarToCartesian(t1_agg, 'cone', 320)

 id_mapping = {}
 for idx, row in t1.iterrows():
 country = row['country']
 key = row['id']
 value = t1_agg[t1_agg['country'] == country]['id'].values[0]
 id_mapping[key] = value

 t2['source'] = t2['source'].map(id_mapping).fillna(t2['source'])
 t2['target'] = t2['target'].map(id_mapping).fillna(t2['target'])
 t2['source'] = t2['source'].astype(int)
 t2['target'] = t2['target'].astype(int)
 t2 = t2[t2['source'] != t2['target']]
 t2['source_target'] = t2.apply(lambda row: tuple(sorted([row['source'], row['target']])), axis=1)
 t2['reverse_relation'] = t2.apply(lambda row: tuple(sorted([row['target'], row['source']])), axis=1)
 t2 = t2.drop_duplicates(subset=['source_target'])
 t2 = t2.drop_duplicates(subset=['reverse_relation'])
 t2 = t2.drop_duplicates(subset=['source', 'target'])
 t2 = t2.drop(columns=['source_target'])
 t2 = t2.drop(columns=['reverse_relation'])

 if len(t1_agg) != 0 and len(t2) != 0:
 t2['cone'] = t2.apply(
 lambda row: max(
 t1_agg.loc[t1_agg['id'] == row['source'], 'cone'].iloc[0], # source cone  t1_agg.loc[t1_agg['id'] == row['target'], 'cone'].iloc[0] # target cone  ),
 axis=1
 )

 return t1_agg, t2


base_dir = './'

# Initialize rankjson and reljson
rankjson = {}
reljson = {}

# Get all unique values of each column in handle_df
for col in handle_df.columns:
 unique_values = handle_df[col].unique() # columnunique values
 rankjson[col] = {}
 reljson[col] = {}
 os.makedirs(f'./local_core/{col}/agg', exist_ok=True)
 # columnunique values
 for value in unique_values:
 # value , JSON  value_str = str(value)

 # Get rank and rel file paths by column name
 rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
 rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

 # read rank rel file
 temp_rank = pd.read_csv(rank_path)
 temp_rel = pd.read_csv(rel_path)

 # Filter data by conditions
 filtered_rank = temp_rank[temp_rank['community'] == value]
 filtered_rel = temp_rel[temp_rel['type'] == str(value)]
 t1, t2 = process_rank_and_rel(filtered_rank, filtered_rel)

 if len(t1) == 1:
 t1.loc[t1.index[0], 'x'] = 1900 / 2
 t1.loc[t1.index[0], 'y'] = 1000 / 2
 t1.loc[t1.index[0], 'r'] = 0
  # filter rankjson reljson , processNaN
 rankjson[col][value_str] = handle_nan(t1.replace({np.nan: None}).to_dict(orient='records'))
 reljson[col][value_str] = handle_nan(t2.replace({np.nan: None}).to_dict(orient='records'))

 with open(f'./local_core/{col}/agg/rank.json', 'w', encoding='utf-8') as rank_file:
 json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

 with open(f'./local_core/{col}/agg/rel.json', 'w', encoding='utf-8') as rel_file:
 json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
 print(f"Saved data for {col}")

### Logic for fdeb
from common.utils.As_rank_rel import fdeb_format

base_dir = './'

# Initialize rankjson and reljson
rankjson = {}
reljson = {}

# Get all unique values of each column in handle_df
for col in handle_df.columns:
 unique_values = handle_df[col].unique() # columnunique values
 rankjson[col] = {}
 reljson[col] = {}
 os.makedirs(f'./local_core/{col}/fdeb', exist_ok=True)
 # columnunique values
 for value in unique_values:
 # value , JSON  value_str = str(value)

 # Get rank and rel file paths by column name
 rank_path = os.path.join(base_dir, f'{col}/handle/rank{col}.csv')
 rel_path = os.path.join(base_dir, f'{col}/handle/rel{col}.csv')

 # read rank rel file
 temp_rank = pd.read_csv(rank_path)
 temp_rel = pd.read_csv(rel_path)

 # Filter data by conditions
 filtered_rank = temp_rank[temp_rank['community'] == value]
 filtered_rel = temp_rel[temp_rel['type'] == str(value)]
 t1, t2 = process_rank_and_rel(filtered_rank, filtered_rel)

 if len(t1) == 1:
 t1.loc[t1.index[0], 'x'] = 1900 / 2
 t1.loc[t1.index[0], 'y'] = 1000 / 2
 t1.loc[t1.index[0], 'r'] = 0

 r1, r2 = fdeb_format(t1, t2)
 # processfdebformatdataNaN
 rankjson[col][value_str] = handle_nan(r1)
 reljson[col][value_str] = handle_nan(r2)
  with open(f'./local_core/{col}/fdeb/rank.json', 'w', encoding='utf-8') as rank_file:
 json.dump(rankjson, rank_file, ensure_ascii=False, indent=4)

 with open(f'./local_core/{col}/fdeb/rel.json', 'w', encoding='utf-8') as rel_file:
 json.dump(reljson, rel_file, ensure_ascii=False, indent=4)
 print(f"Saved data for {col}")


FileNotFoundError: [Errno 2] No such file or directory: './202401/handle/rank202401.csv'